# DeepInfant: Infant Cry Classification

This notebook demonstrates how to train and evaluate the DeepInfant model for infant cry classification using HuggingFace integration.

In [ ]:
import torch
import matplotlib.pyplot as plt
from train import DeepInfantModel, DeepInfantDataset
from torch.utils.data import DataLoader
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np
from train_huggingface import DeepInfantHFModel, DeepInfantHFDataset

## 1. HuggingFace Dataset Preparation

In [ ]:
from datasets import load_dataset, Audio

# Load dataset from HuggingFace (replace with your dataset)
dataset = load_dataset("your_username/your_dataset")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

print(f"Training samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")

## 2. Model Configuration

In [ ]:
from transformers import TrainingArguments, Trainer, AutoConfig

# Configure model
config = AutoConfig.from_pretrained('bert-base-uncased')
config.num_labels = 5  # Number of cry classifications
model = DeepInfantHFModel(config)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_deep_infant_at_end=True,
    push_to_hub=True,
    logging_dir='./logs',
)

## 3. Training

In [ ]:
# Prepare datasets
train_dataset = DeepInfantHFDataset(dataset['train'])
eval_dataset = DeepInfantHFDataset(dataset['validation'])

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train model
trainer.train()

## 4. Evaluation

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

## 5. Push to HuggingFace Hub

In [ ]:
# Push the model to HuggingFace Hub
trainer.push_to_hub()

## 6. Test Predictions

In [ ]:
from predict import InfantCryPredictor

# Load the trained model
predictor = InfantCryPredictor('results/checkpoint-deepinfant')

# Test on a single file
test_file = "path/to/test/audio.wav"
label, confidence = predictor.predict(test_file)
print(f"Prediction: {label} (Confidence: {confidence:.2%})")